# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [12]:
import pandas as pd
import numpy as np
from scipy import stats
import math

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [10]:
w = data[data.race=='w']
b = data[data.race=='b']

# Received a call
b_succ = len(b[b.call == 1])
w_succ = len(w[w.call == 1])

len_b = len(b)
len_w = len(w)

# Proportions
b_prop = 1.0 * b_succ / len_b
w_prop = 1.0 * w_succ / len_w

# CLT calculation
clt = lambda len_df, prop_df: len_df * prop_df > 10 and len_df * (1 - prop_df) > 10

{
    'success_rates': '%s, %s' % (b_succ, w_succ),
    'samples_size': '%s, %s' % (len_b, len_w),
    'proportions': '%s, %s' % (b_prop, w_prop)
}

{'success_rates': '157, 235',
 'samples_size': '2435, 2435',
 'proportions': '0.06447638603696099, 0.09650924024640657'}

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [7]:
clt(len_b, b_prop) and clt(len_w, w_prop)

True

Since CLT for "w" and "b" samples are TRUE, then a normal distribution can be applied. Nevertheless, Fischer's Exact Test can be applied using a contingency table, since the problem comprehends the comparison of 2 proportions.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

In [21]:
from statsmodels.stats.proportion import proportions_ztest as pz

z = 1.96
margin = z * math.sqrt( ( w_prop*(1-w_prop) / len_b) + (b_prop*(1-b_prop)/len_w) )

{
    "Margin of error": margin,
    'confidence_interval': '%s to %s' % (w_prop - b_prop - z * margin, w_prop - b_prop + z*margin),
    'p-value': pz(np.array([b_succ,w_succ]),np.array([len_b,len_w]),value=0)
}

{'Margin of error': 0.015255406349886438,
 'confidence_interval': '0.0021322577636681654 to 0.061933450655223',
 'p-value': (-4.108412152434346, 3.983886837585077e-05)}

The confidence interval does not include 0, but p-value is less than 0.05, hence the null hypothesis can be rejected.

The null hypothesis is: "black and white-sounding names get a similar number of callbacks". False, there is a significant difference, and discrimination preveals.